# SAÉ Labyrinthes



In [4]:
from random import randint

## 3. Implémentation
Définition de la classe `Maze`

In [5]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}

    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def add_wall(self, c1:tuple, c2:tuple) -> None:
        """Méthode qui permet de créer un mur.
        Un mur est entre deux cellules.

        Args:
            c1 (tuple): Cellule d'un côté du mur.
            c2 (tuple): Cellule de l'autre côté du mur.
        
        Returns:
            None: Ne renvoie rien.
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
        return None
            
    def remove_wall(self, c1:tuple, c2:tuple) -> None:
        """Méthode qui permet de supprimer un mur.
        Un mur est entre deux cellules.

        Args:
            c1 (tuple): Cellule d'un côté du mur.
            c2 (tuple): Cellule de l'autre côté du mur.
        
        Returns:
            None: Ne renvoie rien.
        """
        # Suppression du mur
        if c2 not in self.neighbors[c1]:      # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2) # on l'ajoute
        if c1 not in self.neighbors[c2]:      # Si c1 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1) # on l'ajoute
        return None
    
    def get_cells(self) -> list:
        """Méthode qui renvoie une liste de toutes les cellules
        du labyrinthe.

        Returns:
            list: Listes des cellules du labyrinthe.
        """
        L = []
        for i in range(0,self.height):
            for j in range(0, self.width):
                L.append((i,j))
        return L
    
    def inGrid(self, c: tuple) -> bool:
        """Méthode permettant de tester si une cellule
        est dans la grille du labyrinthe ou non.

        Args:
            c (tuple): Cellule observée.

        Returns:
            bool: Renvoie True si elle est dans la grille, False sinon.
        """
        return (0 <= c[0] < self.height) and (0 <= c[1] < self.width)
    
    def get_walls(self) -> list:
        """Méthode qui renvoie une liste de tous les murs du labyrinthe.

        Returns:
            list: Liste de tuples des murs.
        """
        L = []
        lstCells = self.get_cells()
        for c1 in lstCells:
            # Création des voisins de c1, c2 pour l'Est et c3 pour le Sud.
            c2 = (c1[0], c1[1] + 1)
            c3 = (c1[0] + 1, c1[1])
            if self.inGrid(c2) and (c2 not in self.neighbors[c1]):
                L.append((c1,c2))
            if self.inGrid(c3) and (c3 not in self.neighbors[c1]):
                L.append((c1, c3))
        return L
    
    def fill(self) -> None:
        """Méthode qui modifie le labyrinthe et le rend
        complet, avec des murs partout.
        
        Returns:
            None: Ne renvoie rien.
        """
        lstCells = self.get_cells()
        for c1 in lstCells:
            # Création des voisins de c1, c2 pour l'Est et c3 pour le Sud.
            c2 = (c1[0], c1[1] + 1)
            c3 = (c1[0] + 1, c1[1])
            if self.inGrid(c2):
                self.add_wall(c1, c2)
            if self.inGrid(c3):
                self.add_wall(c1, c3)
        return None
    
    def empty(self) -> None:
        """Méthode qui modifie le labyrinthe et le rend
        vide, sans murs.
        
        Returns:
            None: Ne renvoie rien.
        """
        lstCells = self.get_cells()
        for c1 in lstCells:
            # Création des voisins de c1, c2 pour l'Est et c3 pour le Sud.
            c2 = (c1[0], c1[1] + 1)
            c3 = (c1[0] + 1, c1[1])
            if self.inGrid(c2):
                self.remove_wall(c1, c2)
            if self.inGrid(c3):
                self.remove_wall(c1, c3)
        return None
    
    def get_contiguous_cells(self, c:tuple) -> list:
        """Méthode qui renvoie les une liste de cellules contigues
        de 'c' dans la grille.

        Args:
            c (tuple): Cellule cible.

        Returns:
            list: Liste des cellules.
        """
        L = []
        cE = (c[0], c[1] + 1) # Cellule Est (Droite)
        cS = (c[0] + 1, c[1]) # Cellule Sud (Bas)
        cW = (c[0], c[1] - 1) # Cellule West (Gauche)
        cN = (c[0] - 1, c[1]) # Cellule North (Haut)
        # On teste si on peut les placer sur la grille
        if self.inGrid(cE):
            L.append(cE)
        if self.inGrid(cS):
            L.append(cS)
        if self.inGrid(cW):
            L.append(cW)
        if self.inGrid(cN):
            L.append(cN)
        return L
    

    def get_reachable_cells(self, c:tuple) -> list:
        """Méthode qui renvoie les une liste de cellules voisines
        contigues de 'c' dans la grille.

        Args:
            c (tuple): Cellule cible.

        Returns:
            list: Liste des cellules.
        """
        L = []
        cE = (c[0], c[1] + 1) # Cellule Est (Droite)
        cS = (c[0] + 1, c[1]) # Cellule Sud (Bas)
        cW = (c[0], c[1] - 1) # Cellule West (Gauche)
        cN = (c[0] - 1, c[1]) # Cellule North (Haut)
        # On teste si on peut les placer sur la grille
        if self.inGrid(cE) and (cE in self.neighbors[c]):
            L.append(cE)
        if self.inGrid(cS) and (cS in self.neighbors[c]):
            L.append(cS)
        if self.inGrid(cW) and (cW in self.neighbors[c]):
            L.append(cW)
        if self.inGrid(cN) and (cN in self.neighbors[c]):
            L.append(cN)
        return L
    
    
    @classmethod
    def gen_btree(cls, h:int, w:int) -> object:
        """Méthode de classe qui créer un labyrinthe selon l'algorithme
        de l'arbre binaire.

        Args:
            h (int): heigtht, hauteur du labyrinthe créé.
            w (int): width, largeur du labyrinthe créé.

        Returns:
            object: Renvoie labyrinthe déjà créé.
        """
        laby = cls(h, w)
        for i in range(laby.height):
            for j in range(laby.width): # Optimisable avec get_cells() ?
                c = (i, j)
                cE = (c[0], c[1] + 1)
                cS = (c[0] + 1, c[1])
                lstC = [cE, cS]
                # Si cellule Est est dans les voisines condigues de 'c' et que cellule Sud
                # est dans les voisines contigues de 'c', alors on en supprime une des deux au hasard
                if cE in laby.get_contiguous_cells(c) and cS in laby.get_contiguous_cells(c): 
                    laby.remove_wall(c, lstC[randint(0,1)])
                # Si cellule Est est dans les voisines condigues de 'c' mais pas cellule Sud
                # Alors on supprime cellule Est
                elif cE in laby.get_contiguous_cells(c) and cS not in laby.get_contiguous_cells(c):
                    laby.remove_wall(c, cE)
                # Meme principe
                elif cE not in laby.get_contiguous_cells(c) and cS in laby.get_contiguous_cells(c):
                    laby.remove_wall(c, cS)
        return laby

In [57]:
laby = Maze(4, 4)
print(laby.info())
print(laby)

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set()}
- Structure cohérente

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [58]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}

print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [59]:
laby.neighbors[(1,3)].remove((2,3))
laby.neighbors[(2,3)].remove((1,3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [60]:
laby.neighbors[(1, 3)].add((2, 3))
laby.neighbors[(2, 3)].add((1, 3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [19]:
laby.neighbors[(1, 3)].remove((2, 3))
print(laby)
print(laby.info())
laby.neighbors[(2, 3)].remove((1,3))
print(laby.info())

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)}}
- Structure incohérente : (2, 3) X (1, 3)

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1)

In [20]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"Il n'y a pas de mur entre {c1} et {c2} car elles sont mutuellement voisines")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"Il y a un mur entre {c1} et {c2} car {c1} n'est pas dans le voisinage de {c2} et {c2} n'est pas dans le voisinage de {c1}")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

Il y a un mur entre (1, 3) et (2, 3) car (1, 3) n'est pas dans le voisinage de (2, 3) et (2, 3) n'est pas dans le voisinage de (1, 3)


In [21]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"{c1} est accessible depuis {c2} et vice-versa")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"{c1} n'est pas accessible depuis {c2} et vice-versa")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

(1, 3) n'est pas accessible depuis (2, 3) et vice-versa


In [24]:
L = []
for i in range(laby.height):
    for j in range(laby.width):
        L.append((i,j))
print(f"Liste des cellules : \n{L}")

Liste des cellules : 
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


## 4. Manipulation de labyrinthes

In [170]:
laby = Maze(5, 5)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



Ajout de la méthode `remove_wall(c1, c2)` dans la classe `Maze`

In [171]:
laby.remove_wall((0,0),(0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛




Ajout de la méthode `add_wall(c1, c2)` dans la classe `Maze`

In [172]:
laby.add_wall((0,0),(0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



Ajout de la méthode `fill()` dans la classe `Maze`

In [173]:
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



Ajout de la méthode `empty()` dans la classe `Maze`

In [174]:
laby.empty()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [175]:
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



Ajout de la méthode `get_walls()` dans la classe `Maze`

In [176]:
print(laby.get_walls())

[((0, 0), (0, 1)), ((0, 1), (1, 1))]


Ajout de la méthode `get_contiguous_cells(c)` dans la classe `Maze`

In [177]:
print(laby.get_contiguous_cells((0,1)))

[(0, 2), (1, 1), (0, 0)]


Ajout de la méthode `get_reachable_cells(c)` dans la classe `Maze`

In [178]:
print(laby.get_reachable_cells((0,1)))

[(0, 2)]


Ajout de la fonction `get_cells()` dans la classe `Maze`

In [215]:
print(laby.get_cells())

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


## 5. Génération

Nous aborderons les algorithmes de créations de labyrinthes parfaits avec les algo : 
- Par arbre binaire
- De Sidewinder
- Par fusion de chemins
- Par exploration exhaustive
- De Wilson

### 5.1 Arbre binaire

In [12]:
laby = Maze.gen_btree(4, 4)
print(laby)

0 0 [(0, 1), (1, 0)]
0 1 [(0, 2), (1, 1), (0, 0)]
0 2 [(0, 3), (1, 2), (0, 1)]
0 3 [(1, 3), (0, 2)]
1 0 [(1, 1), (2, 0), (0, 0)]
1 1 [(1, 2), (2, 1), (1, 0), (0, 1)]
1 2 [(1, 3), (2, 2), (1, 1), (0, 2)]
1 3 [(2, 3), (1, 2), (0, 3)]
2 0 [(2, 1), (3, 0), (1, 0)]
2 1 [(2, 2), (3, 1), (2, 0), (1, 1)]
2 2 [(2, 3), (3, 2), (2, 1), (1, 2)]
2 3 [(3, 3), (2, 2), (1, 3)]
3 0 [(3, 1), (2, 0)]
3 1 [(3, 2), (3, 0), (2, 1)]
3 2 [(3, 3), (3, 1), (2, 2)]
3 3 [(3, 2), (2, 3)]
┏━━━┳━━━┳━━━┳━━━┓
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃       ┃       ┃
┣━━━╋   ╋━━━╋   ┫
┃           ┃   ┃
┣━━━╋━━━╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛

